https://www.hackerrank.com/challenges/matrix-rotation-algo/problem

Notes: 
+ Upper limits on M and N are 300, so the mat size can be 90,000. Since it's a 2-D matrix, there may be a nested _for_ loop. To avoid O[MXN], I created classes that can compute with less time complexity. 

I create two classes - Loops and MatArrays:
+ Each Matrix is converted into many Loops.
    + Loops are continous lists that hold elements. 
    + Loops are rotated as needed.
    + Lower of the dimensions is always even, so we have continuous Loops.
    + Note that upper limit of number of rotations is 10^6, but it does not matter, since the loop repeats itself after its length. For estimation purposes, T can be reduced to T%len(loop).

+ MatArray 
    + A MatArray holds many Loops.
    + These MatArrays have a dictionary that translates Loop Lists to Matrix Indices.
 


In [1]:
class Loops:
    def __init__(self,mat,l_no):
        self.name=l_no
        self.list = self.makeLoop(mat,l_no)
        
    def makeLoop(self,mat,l_no):
        #M=mat.M
        M=len(mat)
        #N=mat.N
        N=len(mat[0])
        l_loop=[]
        '''
        M=4
        N=6      
          0   1   2   3   4   5 
        0 X   X   X   X   X   X M-4
        1 X                   X M-3
        2 X                   X M-2
        3 X   X   X   X   X   X M-1
         N-6 N-5 N-4 N-3 N-2 N-1
        '''
        # first row  (all)
        l_loop+=mat[0+l_no][l_no:N-l_no]
            
        # last column (trimmed)
        for r in range(1+l_no,M-1-l_no,1): # 1 to M-2 for l_no=0
            l_loop.append(mat[r][N-1-l_no])
        
        # last row (all, reversed)
        l_row=mat[M-1-l_no][0+l_no:N-l_no]
        l_row.reverse()
        l_loop+=l_row
        
        # first column (trimmed, reversed using the range)
        for r in range(M-2-l_no,0+l_no,-1): # M-2 to 1 for l_no=0
            l_loop.append(mat[r][0+l_no])
            
        return l_loop
    
    def getList(self):
        return self.list
    
    def setList(self,l):
        self.list=l
        
    def rotateLoop(self,R):
        li=self.getList()
        if R==0: 
            self.setList(li)
        else: # R<0 or R>0
            R=R%len(li)
            self.setList(li[R:]+li[:R])
        
class MatArray:
    def __init__(self,mat):
        self.M=len(mat)
        self.N=len(mat[0])
        self.n_loops=int(min(self.M,self.N)/2)
        self.loops=self.makeAllLoops(mat)
        self.matLoopDict=self.makeDict()
        self.matrix=self.makeMatrix()
        
    def makeAllLoops(self,mat):
        loops={}
        n_loops=self.n_loops
        for i in range(n_loops):
            loops[i]=Loops(mat,i)
        return loops
 
    def __iter__(self):
        return iter(self.loops.values())
    
    def rotateMatrix(self,R): # returns a new matrix, original  matrix is unchanged
        newMat=MatArray(self.matrix)
        for loop in newMat:
            loop.rotateLoop(R)
        newMat.matrix=newMat.makeMatrix()
        return newMat
    
    def makeDict(self): 
        '''
        N=4
        M=5
       N-4 N-3 N-2 N-1
        0   1   2   3
        X   X   X   X    M-5 
    13  X           X 4  M-4  
    12  X           X 5  M-3  
    11  X           X 6  M-2  
        X   X   X   X    M-1
       10   9   8   7
    
        '''
        n_loops=self.n_loops
        M=self.M
        N=self.N
        d={}
        # Form: d[(r,c)] = (l_no,idx)
        for i in range(n_loops):
            #first row
            c_i=0
            for l_i in range(N-(2*i)):
                d[(i,i+c_i)]=(i,l_i)
                c_i+=1
                
            # last col
            r_i=0
            for l_i in range(N-(2*i),N+M-2-(4*i)):
                d[(i+1+r_i,N-1-i)]=(i,l_i)
                r_i+=1
                
            # last row
            c_i=0
            for l_i in range(N+M-2-(4*i),(2*N)+M-2-(6*i)-1):
                d[(M-1-i,N-1-i-c_i)]=(i,l_i)
                c_i+=1
                
            # first col
            r_i=0
            for l_i in range((2*N)+M-2-(6*i)-1,(2*N)+(2*M)-4-(8*i)):
                d[(M-1-i-r_i,i)]=(i,l_i)
                r_i+=1
        return d
    
    def makeMatrix(self):
        N=self.N
        M=self.M
        mat=[]
        d=self.matLoopDict
        d_loops=self.loops
        
        for r in range(M):
            mat.append([])
            for c in range(N):
                mat[r].append(0)
        for k in d.keys():
            r=k[0]
            c=k[1]
            l_n=d[k][0]
            l=d_loops[l_n].getList()
            i=d[k][1]
            mat[r][c]=l[i]
        return mat

In [2]:
def matrixRotation(mat,r):
    MA=MatArray(mat)
    answer=MA.rotateMatrix(r).matrix

    for row in answer:
        s=''
        for i in row:
            s+=str(i)+' '
        print(s)

In [3]:
in_list=['4 4 1',
         '1 2 3 4',
         '5 6 7 8',
         '9 10 11 12',
         '13 14 15 16']

In [4]:
params=[int(i) for i in in_list[0].split()]
M=params[0]
N=params[1]
R=params[2]
mat=[]
for i in range(1,M+1,1):
    mat.append([int(i) for i in in_list[i].split()])

In [5]:
matrixRotation(mat,2)

3 4 8 12 
2 11 10 16 
1 7 6 15 
5 9 13 14 
